### Supplementary Figures ###
This notebooks contains code used to make supplementary figures for the manuscript using the detrital zircon data.

In [ ]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import rc
rc("pdf", fonttype=42)

from geoscripts.dz import dz


In [ ]:
# Load samples using geoscripts

samples_published = dz.load_all()

samples_new = dz.load_all('data_processing/dz_rel_age_filter/')

samples = samples_published + samples_new

In [ ]:
# Add shorthand name for newly-published samples

nickname_dic = {'CT15076':'KZ1','CT15082':'KZ2','CT15092':'KZ3','CT15127':'KZ4','CT15099':'KZ5','CT15113':'KZ6','AB0913':'RU1','CT130918-2A':'SV1',
'100211-9A':'SV2','CT130918-9A':'SV3','CT130919-5A':'SV4','100411-5':'SV5','CT130919-8A':'SV6','AB0926':'SV7','100211-1A':'SV8'}

for smp in samples:
    if smp.name in nickname_dic:
        smp.original = smp.name
        smp.name = nickname_dic[smp.name]

        # Purge numbers <1 that shouldn't be in the age column
        smp.bestage = smp.bestage[smp.bestage>1]

        print(smp.name,smp.original)

In [ ]:
# Organize new samples into blocks

kz_samples = [smp for smp in samples if 'KZ' in smp.name]
kz_samples.sort(key=lambda x: x.name, reverse=True)
sv_samples = [smp for smp in samples if 'SV' in smp.name]
sv_samples.sort(key=lambda x: x.name, reverse=True)

ru = [smp for smp in samples if 'RU' in smp.name]

In [ ]:
# Plot MDA ages for Svaneti samples
fig, axs = plt.subplots(4,2,dpi=300,figsize=(6.5,9))

for k,smp in enumerate(sv_samples):
    smp.plot_mda(axs.flat[k],syst_error=True)

plt.tight_layout()

fig.savefig('sv_mda_plots.pdf')

In [ ]:
# Create function for pie chart labeling
def my_autopct(pct):
    return ('%1.0f%%' % pct) if pct > 1 else ''

In [ ]:
# Plot pie charts for Svaneti samples
fig, axs = plt.subplots(4,2,dpi=300,figsize=(6.5,9))

# tiny version
fig2, axs2 = plt.subplots(8,1,dpi=300,figsize=(0.5,5))

bins = [65,145,174,184,200,250,360,420,540,4000]

spans = [(150,190),(220,260),(260,340),(410,490),(550,700),(1500,2300),(2400,2800)]
colors = ['#EE7733', '#0077BB', '#EE3377','#009988', '#CC3311', '#33BBEE', '#BBBBBB','white']

textprops = {"fontsize":6}

for k,smp in enumerate(sv_samples):
    sizes = []

    for span in spans:
        condition = (smp.bestage>=span[0])&(smp.bestage<span[1])
        count = smp.bestage[condition].count()
        sizes.append(count)
    
    not_counted = len(smp.bestage)-sum(sizes)

    sizes_total = sizes+[not_counted]

    axs.flat[k].pie(sizes_total,colors=colors,autopct=my_autopct,textprops=textprops,wedgeprops={"alpha": 0.5})
    axs2.flat[k].pie(sizes_total,colors=colors,wedgeprops={"alpha": 0.5})


    axs.flat[k].set_title(smp.name)

axs.flat[-1].legend(labels=spans, title='Age Ranges (Ma)')
plt.tight_layout()

fig.savefig('sv_pie.pdf')
fig2.savefig('sv_pie_tiny.pdf')

In [ ]:
# Plot MDA ages for Kazbegi samples
fig, axs = plt.subplots(4,2,dpi=300,figsize=(6.5,9))

for k,smp in enumerate(kz_samples+ru):
    smp.plot_mda(axs.flat[k],syst_error=True)

axs.flat[7].remove()
plt.tight_layout()

fig.savefig('kz_mda_plots.pdf')

In [ ]:
# Plot pie charts for Kazbegi samples
fig, axs = plt.subplots(4,2,dpi=300,figsize=(6.5,9))

# tiny version
fig2, axs2 = plt.subplots(8,1,dpi=300,figsize=(0.5,5))

spans = [(150,190),(220,260),(260,340),(410,490),(550,700),(1500,2300),(2400,2800)]
colors = ['#EE7733', '#0077BB', '#EE3377','#009988', '#CC3311', '#33BBEE', '#BBBBBB','white']

textprops = {"fontsize":6}

for k,smp in enumerate(kz_samples+ru):
    sizes = []

    for span in spans:
        condition = (smp.bestage>=span[0])&(smp.bestage<span[1])
        count = smp.bestage[condition].count()
        sizes.append(count)
    
    not_counted = len(smp.bestage)-sum(sizes)

    sizes_total = sizes+[not_counted]

    axs.flat[k].pie(sizes_total,colors=colors,autopct=my_autopct,textprops=textprops,wedgeprops={"alpha": 0.5})
    axs2.flat[k].pie(sizes_total,colors=colors,wedgeprops={"alpha": 0.5})

    axs.flat[k].set_title(smp.name)

axs.flat[7].remove()
axs2.flat[7].remove()
axs.flat[6].legend(labels=spans, title='Age Ranges (Ma)')
plt.tight_layout()

fig.savefig('kz_pie.pdf')
fig2.savefig('kz_pie_tiny.pdf')

In [ ]:
# Redo MDAs for previously published samples from Cowgill16 and Vasey20

syst_238 = {'NW-GC':0.9,'N1':1.0,'N2':0.8,'N3':0.8,'N5':0.9}

# Need to change discordance filter age to 600?
for smp in samples:
    if smp.name in ['NW-GC','N1','N2','N3','N5']:
        smp.calc_bestage(col_238='238/206 U-Pb Age',col_207='206/207 Pb Age',err_238='238 Error',err_207='206 Error',use_err=True,err_lev='1sig',
        disc_cutoff=20,disc_age_cutoff=600)
        smp.convert_1sigto2sig()
        smp.syst_238 = syst_238[smp.name]
        smp.calc_mda(systematic=True)
        age_errors = pd.concat([smp.bestage,smp.besterror],axis=1).dropna(how='any')
        ages_sorted = age_errors.sort_values(by=['Best Age'],ignore_index=True)
        smp.ysg = ages_sorted.iloc[0,0]
        smp.ysg_err = ages_sorted.iloc[0,1]

In [ ]:
# Plot MDAs for Cowgill16 and Vasey20 samples and save as a figure
fig, axs = plt.subplots(3,2,dpi=300,figsize=(6.5,9*0.75))

old_smps = [smp for smp in samples if smp.name in ['NW-GC','N1','N2','N3','N5']]

for k,smp in enumerate(old_smps):
    smp.plot_mda(axs.flat[k],syst_error=True)

axs.flat[5].remove()
plt.tight_layout()

fig.savefig('old_mda_plots.pdf')

In [ ]:
# Define plotting function for column of DZ analyses

def plot_dz_columns(smps,conc=None,save=False,filename=None,plot_mapped=False,jcarb=False,mpeaks=False,**kwargs):
    
    # Set up axes, title, limits, and plot KDE
    
    nplots = len(smps)
    height = nplots*1.5
    
    # If only 1 samples input
    if len(smps)==1:
        fig,axs = plt.subplots(nplots,dpi=300,figsize=(7.5,height))
        smp = smps[0]
        smp.kde(ax=axs,**kwargs)
        axs.set_title(smp.name)
        axs.set_xlim(50,4000)

        # Add MDA information to plot
        if mda:

            mla_data = pd.read_csv('data_processing/MLA_no_filter/'+smp.original+'_mla.csv',index_col=0)
            smp.mla = mla_data.loc['t'].values[0]
            smp.mla_err = mla_data.loc['s[t]'].values[0]*2 # Convert 1s to 2s

            mla_text = 'MLA: ' + str(round(smp.mla,1)) + ' ± ' + str(round(smp.mla_err,1))
            axs[k].text(0.22,0.3,mla_text,transform=axs[k].transAxes)
    
    # If multiple samples input
    else:
        fig,axs = plt.subplots(nplots,dpi=300,figsize=(7.5,height),sharex=True)
        for k,smp in enumerate(smps):
            smp.kde(ax=axs[k],**kwargs)
            axs[k].set_title(smp.name)
            axs[k].set_xlim(50,4000)
            
            # Add MDA information to plot
            if conc==True:
                mla_data = pd.read_csv('data_processing/MLA_no_filter/'+smp.original+'_mla.csv',index_col=0)
                smp.mla = mla_data.loc['t'].values[0]
                smp.mla_err = mla_data.loc['s[t]'].values[0]*2 # Convert 1s to 2s

                mla_text = 'MLA: ' + str(round(smp.mla,1)) + ' ± ' + str(round(smp.mla_err,1))
                axs[k].text(0.02,0.3,mla_text,transform=axs[k].transAxes)
            
            if conc==False:
                mla_data = pd.read_csv('data_processing/MLA_rel_age_filter/'+smp.original+'_mla.csv',index_col=0)
                smp.mla = mla_data.loc['t'].values[0]
                smp.mla_err = mla_data.loc['s[t]'].values[0]*2 # Convert 1s to 2s

                mla_text = 'MLA: ' + str(round(smp.mla,1)) + ' ± ' + str(round(smp.mla_err,1))
                axs[k].text(0.02,0.3,mla_text,transform=axs[k].transAxes)

            # Add mapped depositional age as a gray bar
            if plot_mapped:
                trans = axs[k].get_xaxis_transform()
                axs[k].plot([smp.mapped_age[0],smp.mapped_age[1]],[0.02,0.02],color='gray',linewidth=2,clip_on=False,transform=trans)
                axs[k].set_ylim(bottom=0)

            # Add Early Jurassic and Late Carboniferous boxes
            if jcarb:
                ejmin = 174.1
                ejmax = 201.3

                lcarbmin = 298.9
                lcarbmax = 323.2

                axs[k].axvline(ejmax,linestyle='--',color='navy',zorder=0)
                axs[k].axvline(ejmin,linestyle='--',color='navy',zorder=0)
                axs[k].axvline(lcarbmax,linestyle='--',color='purple',zorder=0)
                axs[k].axvline(lcarbmin,linestyle='--',color='purple',zorder=0)
            
            # Add lines for major peaks
            if mpeaks:
                peaks = [170,240,300,450,2000,2600]
                for peak in peaks:
                    axs[k].axvline(peak,linestyle='--',color='darkorange',zorder=0)
                
    plt.tight_layout()
    
    # Save the figure as a file
    if save == True:
        fig.savefig(filename)
    
    return

In [ ]:
# Plot individual KDEs for additional samples used in Figure 7 (composite KDEs over time)

dizi_names = ['Khelra','WGC-2']
idliani_names = ['Khopuri']
jvari_names = ['SW-GC','WGC-3','CT130924-9A']
gvpas_names = ['Tskhradzmula','CGC-1']
ana_names = ['CGC-2']

lc_names = ['LC-1','LC-2','LC-3','TC-1','Tovuz']

dizi_smps = [smp for smp in samples if smp.name in dizi_names]
idliani_smps = [smp for smp in samples if smp.name in idliani_names]
jvari_smps = [smp for smp in samples if smp.name in jvari_names]
gvpas_smps = [smp for smp in samples if smp.name in gvpas_names]
ana_smps = [smp for smp in samples if smp.name in ana_names]
lc_smps = [smp for smp in samples if smp.name in lc_names]

for smp in lc_smps:
    smp.bestage = smp.bestage[smp.bestage>65]

block = dizi_smps + idliani_smps + jvari_smps + gvpas_smps + ana_smps + lc_smps

ticks = [50,100,150,200,300,500,1000,2000,3000]
plot_dz_columns(block,bw_adjust=0.2,ticks=ticks,save=True,filename='supp_kdes.pdf')

In [ ]:
# Load samples using concordia distance filter

smps_conc = dz.load_all('data_processing/dz_concdist_filter/')

sv_conc = []
kz_conc = []

map_ages = pd.read_csv('map_ages.csv')

color_dict = {'J1':'midnightblue','J2':'blue','J3':'skyblue','K1':'darkgreen',
              'K2':'chartreuse'}

for smp in smps_conc:

    map_index = map_ages[map_ages['Sample Number']==smp.name].index
    smp.reported_age = map_ages.loc[map_index,'Reported Age'].values[0]
    smp.color = color_dict[smp.reported_age]

    smp.original = smp.name
    smp.name = nickname_dic[smp.name]
    print(smp.name)

    if smp.name=='SV1':
        smp.color='red'
    elif smp.name in ['SV2','SV3']:
        smp.color='purple'
    elif smp.name in ['SV7']:
        smp.color='skyblue'
    elif smp.name=='KZ1':
        smp.color='skyblue'


    if 'SV' in smp.name:
        sv_conc.append(smp)

    if ('KZ' in smp.name)|('RU' in smp.name):
        kz_conc.append(smp)


kz_conc.sort(key=lambda x: x.name, reverse=True)
sv_conc.sort(key=lambda x: x.name, reverse=True)


In [ ]:
# Plot SV samples
plot_dz_columns(sv_conc,conc=True,bw_adjust=0.2,ticks=ticks,save=True,filename='sv_kdes_conc.pdf')

In [ ]:
# Plot KZ samples
plot_dz_columns(kz_conc,conc=True,bw_adjust=0.2,ticks=ticks,save=True,filename='kz_kdes_conc.pdf')

In [ ]:
# Load samples using relative age filter to compare with concordia distance filter

smps_rel = dz.load_all('data_processing/dz_rel_age_filter/')

sv_rel = []
kz_rel = []

map_ages = pd.read_csv('map_ages.csv')

color_dict = {'J1':'midnightblue','J2':'blue','J3':'skyblue','K1':'darkgreen',
              'K2':'chartreuse'}

for smp in smps_rel:

    map_index = map_ages[map_ages['Sample Number']==smp.name].index
    smp.reported_age = map_ages.loc[map_index,'Reported Age'].values[0]
    smp.color = color_dict[smp.reported_age]

    smp.original = smp.name
    smp.name = nickname_dic[smp.name]
    print(smp.name)

    if smp.name=='SV1':
        smp.color='red'
    elif smp.name in ['SV2','SV3']:
        smp.color='purple'
    elif smp.name in ['SV7']:
        smp.color='skyblue'
    elif smp.name=='KZ1':
        smp.color='skyblue'


    if 'SV' in smp.name:
        sv_rel.append(smp)

    if ('KZ' in smp.name)|('RU' in smp.name):
        kz_rel.append(smp)


kz_rel.sort(key=lambda x: x.name, reverse=True)
sv_rel.sort(key=lambda x: x.name, reverse=True)

In [ ]:
# Plot SV samples
plot_dz_columns(sv_rel,conc=False,bw_adjust=0.2,ticks=ticks,save=True,filename='sv_kdes_rel.pdf')

In [ ]:
# Plot KZ samples
plot_dz_columns(kz_rel,conc=False,bw_adjust=0.2,ticks=ticks,save=True,filename='kz_kdes_rel.pdf')